# SRResnet-GAN

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, PReLU, LeakyReLU, Layer, Conv2D, BatchNormalization, Flatten
from tensorflow.keras.applications.vgg16 import VGG16

## Residual Block

In [4]:
class ResidualBlock(Layer):
    def __init__(self, channel=64, kernel_size=(3, 3)):
        super().__init__()
        self.conv1 = Conv2D(channel, kernel_size, padding='same', use_bias=False)
        self.bn1 = BatchNormalization()
        self.prelu = PReLU()
        self.conv2 = Conv2D(channel, kernel_size, padding='same', use_bias=False)
        self.bn2 = BatchNormalization()

    def call(self, x, training=None, mask=None):
        h = self.prelu(self.bn1(self.conv1(x), training))
        return x + self.bn2(self.conv2(h), training)

## Conv-Bn-Relu Block

In [5]:
class ConvBnLReluBlock(Layer):
    def __init__(self, channel=64, kernel_size=(3, 3)):
        super().__init__()
        self.conv = Conv2D(channel, kernel_size, padding='same', use_bias=False)
        self.bn = BatchNormalization()
        self.lrelu = LeakyReLU()

    def call(self, x, training=None, mask=None):
        return self.lrelu(self.bn(self.conv(x), training))

## Generator (SRResnet)

In [6]:
class Generator(Model):
    def __init__(self, channel=64, num_resblock=5):
        super().__init__()
        self.conv1 = Conv2D(channel, (9, 9), padding='same')
        self.prelu1 = PReLU()
        
        self.resblock_list = [ResidualBlock(channel, (3, 3)) for _ in range(num_resblock)]
        
        self.conv2 = Conv2D(channel, (3, 3), padding='same', use_bias=False)
        self.bn = BatchNormalization()
        self.prelu2 = PReLU()
        
        self.conv3 = Conv2D(channel * 4, (3, 3), padding='same')
        self.prelu3 = PReLU()
        
        self.conv4 = Conv2D(channel * 4, (3, 3), padding='same')
        self.prelu4 = PReLU()
        self.conv5 = Conv2D(3, (9, 9), padding='same')

    def call(self, x, training=None, mask=None):
        h = self.prelu1(self.conv1(x))
        h_skip = h
        
        for resblock in self.resblock_list:
            h = resblock(h, training)
        
        h = self.prelu2(self.bn(self.conv2(h), training))
        h = h + h_skip
        
        h = self.prelu3(tf.nn.depth_to_space(self.conv3(h), 2))
        h = self.prelu4(tf.nn.depth_to_space(self.conv4(h), 2))
        return self.conv5(h)

## Discriminator

In [7]:
class Discriminator(Model):
    def __init__(self, channel=64):
        super().__init__()
        self.conv1 = Conv2D(channel, (3, 3), padding='same')
        self.lrelu1 = LeakyReLU()
        
        self.block_list = list()
        self.block_list.append(ConvBnLReluBlock(channel))
        self.block_list.append(ConvBnLReluBlock(channel * 2))
        self.block_list.append(ConvBnLReluBlock(channel * 2))
        self.block_list.append(ConvBnLReluBlock(channel * 4))
        self.block_list.append(ConvBnLReluBlock(channel * 4))
        self.block_list.append(ConvBnLReluBlock(channel * 8))
        self.block_list.append(ConvBnLReluBlock(channel * 8))
        
        self.flatten = Flatten()
        self.dense1 = Dense(1024)
        self.lrelu2 = LeakyReLU()
        self.dense2 = Dense(1, activation='sigmoid')

    def call(self, x, training=None, mask=None):
        h = self.lrelu1(self.conv1(x))
        for block in self.block_list:
            h = block(h, training)
        h = self.flatten(h)
        h = self.lrelu2(self.dense1(h))
        return self.dense2(h)

## Dataset (Caltech101)

In [8]:
dataset = tfds.load(name='caltech101', split='train')
dataset = dataset.map(lambda x: (tf.image.resize(tf.cast(x['image'], tf.float32), (8, 8), tf.image.ResizeMethod.BICUBIC) / 255.0,
                                 tf.image.resize(tf.cast(x['image'], tf.float32), (32, 32), tf.image.ResizeMethod.BICUBIC) / 255.0)).batch(1)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

## VGG Model, Hyperparameters

In [ ]:
generator = Generator(16)
discriminator = Discriminator(16)

vgg = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
vgg = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
vgg.trainable = False

w_gan = 1e-2
w_vgg = 1e-5

optim_d = tf.optimizers.Adam(1e-4)
optim_g = tf.optimizers.Adam(1e-4)

d_mean = tf.metrics.Mean()
g_mean = tf.metrics.Mean()
vgg_mean = tf.metrics.Mean()
l1_mean = tf.metrics.Mean()

## Losses

In [ ]:
@tf.function
def l1_loss_func(y, y_):
    return tf.reduce_mean(tf.abs(y - y_))

@tf.function
def vgg_loss_func(y, y_):
    return tf.reduce_mean((vgg(y) - vgg(y_)) ** 2)

@tf.function
def discriminator_loss(real, fake):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real), real)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake), fake)
    return real_loss + fake_loss

@tf.function
def generator_loss(fake):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake), fake)


## Training Step

In [ ]:
@tf.function
def train_step(image_lr, image_hr, optim_d, optim_g):
    with tf.GradientTape() as tape_d, tf.GradientTape() as tape_g:
        image_sr = generator(image_lr, training=True)
        
        d_real = discriminator(image_hr, training=True)
        d_fake = discriminator(image_sr, training=True)
        
        d_loss = discriminator_loss(d_real, d_fake)
        g_loss = generator_loss(d_fake)
        
        vgg_loss = vgg_loss_func(image_hr, image_sr)
        l1_loss = l1_loss_func(image_hr, image_sr)
        
        loss = w_gan * g_loss + w_vgg * vgg_loss + l1_loss
        
        gradients_d = tape_d.gradient(d_loss, discriminator.trainable_weights)
        gradients_g = tape_g.gradient(loss, generator.trainable_weights)
    
    optim_d.apply_gradients(zip(gradients_d, discriminator.trainable_weights))
    optim_g.apply_gradients(zip(gradients_g, generator.trainable_weights))
    return d_loss, g_loss, vgg_loss, l1_loss

## Training Loop

In [ ]:
for epoch in range(100):
    for img_lr, img_hr in dataset:
        d_loss, g_loss, vgg_loss, l1_loss = train_step(img_lr, img_hr, optim_d, optim_g)

        d_mean.update_state(d_loss)
        g_mean.update_state(g_loss)
        vgg_mean.update_state(vgg_loss)
        l1_mean.update_state(l1_loss)

    print('epoch: {}, d_loss: {}, g_loss: {}, vgg_loss: {}, l1_loss: {}'.format(epoch+1,
                                                                d_mean.result(),
                                                                g_mean.result(),
                                                                vgg_mean.result(),
                                                                l1_mean.result()))
    img_sr_list = list()
    img_lr_list = list()
    img_hr_list = list()
    for img_lr, img_hr in dataset.take(10):
        img_sr = generator(img_lr)
        
        img_lr_list.append(tf.image.resize(img_lr[0], (32, 32),
                            method=tf.image.ResizeMethod.NEAREST_NEIGHBOR))
        img_sr_list.append(img_sr[0])
        img_hr_list.append(img_hr[0])
    
    img_lr = np.concatenate(img_lr_list, axis=1)
    img_sr = np.concatenate(img_sr_list, axis=1)
    img_hr = np.concatenate(img_hr_list, axis=1)
    img = np.concatenate([img_lr, img_sr, img_hr], axis=0)
    
    plt.imshow(img)
    plt.show()

    d_mean.reset_states()
    g_mean.reset_states()
    vgg_mean.reset_states()
    l1_mean.reset_states()